# HU Extension              Assignment 07       E63 Big Data Analytics 		                  	
### Handed out: 03/10/2023                                              Due by 6PM EST, 03/25/2023
### Emmanuel Aboah

### Problem 1. 
Install Cassandra, a NoSQL Database. Depending on your operating system follow instructions on http://cassandra.apache.org or follow instructions in the notes for Lecture 07 or notes for Lab 06. Demonstrate that you can start and stop Cassandra server. Open CQLSH prompt and execute any CQL command. Capture the output on the server console.

I am us a [datstax enterprise image](https://hub.docker.com/r/datastax/dse-server) to run my cassandra cluster.

```bash
# Pull dse-server version 6.8.25
(base) manny@LAPTOP-85L1BUVJ:~$ docker pull datastax/dse-server:6.8.25
6.8.25: Pulling from datastax/dse-server
f003217c5aae: Pull complete
9a93b586a561: Pull complete
ef9d705407f4: Pull complete
a03fec40d050: Pull complete
7c961e16151e: Pull complete
a6f6494b8e4e: Pull complete
d2db2b54fa49: Pull complete
9a9ee239f251: Pull complete
da0747a07fdb: Pull complete
7679539bd51d: Pull complete
17dd0a0d4dfd: Pull complete
42c8fc9a6c23: Pull complete

# Run the container
docker run -e DS_LICENSE=accept --name my-dse -d datastax/dse-server:6.8.25
9314c45eb8ea492f192a213b1de1f575b71baeb41febef851b1f73c80dff9df3

# Exec into cqlsh
 docker exec -it my-dse cqlsh
Connected to Test Cluster at 127.0.0.1:9042.
[cqlsh 6.8.0 | DSE 6.8.25 | CQL spec 3.4.5 | DSE protocol v2]
Use HELP for help.

# Describe keyspaces
cqlsh> describe keyspaces;

system_virtual_schema  system_schema  dse_leases          system_traces
dse_system_local       system_auth    system_backups      dse_perf
dse_security           system_views   dse_insights        dse_insights_local
solr_admin             system         system_distributed  dse_system

# Stop the server
(base) manny@LAPTOP-85L1BUVJ:~$ docker stop my-dse
my-dse
(base) manny@LAPTOP-85L1BUVJ:~$ docker ps
CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES
(base) manny@LAPTOP-85L1BUVJ:~$

# Start server
(base) manny@LAPTOP-85L1BUVJ:~$ docker start my-dse
my-dse

# Nodetool status
(base) manny@LAPTOP-85L1BUVJ:~$ docker exec -it my-dse nodetool status
Datacenter: dc1
===============
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving/Stopped
--  Address     Load       Owns (effective)  Host ID                               Token                                    Rack
UN  172.17.0.2  200.31 KiB  100.0%            1cea3ae0-7469-4014-a78a-b8b9baab7bf0  4235639552853138822                      rack1
```

### Problem 2. 
Using cqlsh tool, create new keyspace and a new table Employees which will contain information about employees of a company. Every employee should have an employee Id, first name, last name, office id, up to 3 phone number and the full name of his or her supervisor. Supervisor field could be null. Populate that table with 5 employees working in two different offices. Similarly, create table Offices which will keep track of office name, city, street address, office id. Is there a way to impose referential integrity between table Employees and table Offices. Populate your table with at least 3 different offices. Two office should have current employees. Demonstrate that you can retrieve selected rows in either table. You can make selection by specifying primary keys for either table or by specifying values of some the other attributes. Could you, using a single CQL query, select information about an employee’s last and first name and the office in which he or she works. This problem might require some light reading of Cassandra’s User Guide.

CQL script to create tables and load data.<br>

/employees.cql

```sql
CREATE KEYSPACE hw07 WITH replication = { 'class': 'SimpleStrategy', 'replication_factor': '2' };

use hw07;

CREATE TABLE IF NOT EXISTS employees (
    employeeId uuid,
    firstName TEXT,
    lastName TEXT,
    officeId TEXT,
    phoneNumbers SET<TEXT>,
    supervisorName TEXT,
    primary key (employeeId, officeId)
);

BEGIN BATCH
INSERT INTO employees (employeeId, firstName, lastName, officeId, phoneNumbers, supervisorName)
VALUES (uuid(), 'Manny', 'Aboah', '0056', {'572-125-8898', '804-569-2235', '703-895-4563'}, 'Sherry Williams')
INSERT INTO employees (employeeId, firstName, lastName, officeId, phoneNumbers, supervisorName)
VALUES (uuid(), 'Allissa', 'Abdelwahed', '0057', {'571-145-6698', '804-339-2555', '703-892-4565'}, 'Priya Singh')
INSERT INTO employees (employeeId, firstName, lastName, officeId, phoneNumbers, supervisorName)
VALUES (uuid(), 'Johny', 'Quest', '0056', {'571-125-8899', '804-222-3355', '703-333-1111'}, 'Joe Buck')
INSERT INTO employees (employeeId, firstName, lastName, officeId, phoneNumbers, supervisorName)
VALUES (uuid(), 'Danny', 'Boyd', '0056', {'572-265-6969', '804-555-1232', '703-888-0036'}, 'Tucker Tank')
INSERT INTO employees (employeeId, firstName, lastName, officeId, phoneNumbers, supervisorName)
VALUES (uuid(), 'Rebel', 'Wilson', '0057', {'572-777-8898', '804-111-2235', '703-123-2244'}, 'Sherry Williams')
APPLY BATCH;

CREATE TABLE IF NOT EXISTS offices (
    officeName TEXT,
    city TEXT,
    streetAddress TEXT,
    officeId TEXT,
    primary key (officeId, officeName)
);

BEGIN BATCH
INSERT INTO offices (officeName, city, streetAddress, officeId)
VALUES ('Deloitte Rosslyn', 'Rosslyn', '1777 N Kent St, Arlington, VA 22209', '0056')
INSERT INTO offices (officeName, city, streetAddress, officeId)
VALUES ('AMZ-10', 'Reston', '1777 Worldgate Dr. Reston, VA 20171', '0057')
INSERT INTO offices (officeName, city, streetAddress, officeId)
VALUES ('Vaco', 'Falls Church', '2980 Fairview Park Dr, Falls Church, VA 22042', '0058')
INSERT INTO offices (officeName, city, streetAddress, officeId)
VALUES ('Deloitte Tysons', 'Tysons Corner', '7900 Tysons One Pl #800, McLean, VA 22102', '0059')
APPLY BATCH;
```

```bash
# Copy the script to my-dse server/ cluster
(base) manny@LAPTOP-85L1BUVJ:~/dev/cscie-63/hw07$ docker cp employees.cql my-dse:/opt/dse

# Apply scripts via cqlsh
dse@9314c45eb8ea:~$ cqlsh -f employees.cql

# Describe keyspaces
cqlsh> describe keyspaces;

system_virtual_schema  system_auth    system_backups      dse_insights_local
hw07                   system_views   dse_insights        dse_system
cscie63_hw07           system         system_distributed  dse_security
solr_admin             system_traces  dse_system_local
system_schema          dse_leases     dse_perf

# Describe hw07 keyspace's tables
cqlsh> describe keyspace hw07;

CREATE KEYSPACE hw07 WITH replication = {'class': 'SimpleStrategy', 'replication_factor': '2'}  AND durable_writes = true;

CREATE TABLE hw07.employees (
    employeeid uuid,
    officeid text,
    firstname text,
    lastname text,
    phonenumbers set<text>,
    supervisorname text,
    PRIMARY KEY (employeeid, officeid)
) WITH CLUSTERING ORDER BY (officeid ASC)
    AND additional_write_policy = '99PERCENTILE'
    AND bloom_filter_fp_chance = 0.01
    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
    AND comment = ''
    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}
    AND compression = {'chunk_length_in_kb': '64', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
    AND crc_check_chance = 1.0
    AND default_time_to_live = 0
    AND gc_grace_seconds = 864000
    AND max_index_interval = 2048
    AND memtable_flush_period_in_ms = 0
    AND min_index_interval = 128
    AND nodesync = {'enabled': 'true', 'incremental': 'true'}
    AND read_repair = 'BLOCKING'
    AND speculative_retry = '99PERCENTILE';

CREATE TABLE hw07.offices (
    officeid text,
    officename text,
    city text,
    streetaddress text,
    PRIMARY KEY (officeid, officename)
) WITH CLUSTERING ORDER BY (officename ASC)
    AND additional_write_policy = '99PERCENTILE'
    AND bloom_filter_fp_chance = 0.01
    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
    AND comment = ''
    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}
    AND compression = {'chunk_length_in_kb': '64', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
    AND crc_check_chance = 1.0
    AND default_time_to_live = 0
    AND gc_grace_seconds = 864000
    AND max_index_interval = 2048
    AND memtable_flush_period_in_ms = 0
    AND min_index_interval = 128
    AND nodesync = {'enabled': 'true', 'incremental': 'true'}
    AND read_repair = 'BLOCKING'
    AND speculative_retry = '99PERCENTILE';

# Show data from eployees table
cqlsh> select * from hw07.employees;

 employeeid                           | officeid | firstname | lastname   | phonenumbers                                     | supervisorname
--------------------------------------+----------+-----------+------------+--------------------------------------------------+-----------------
 7689cb09-dc2a-4b09-9020-d75fcfae4baf |     0056 |     Johny |      Quest | {'571-125-8899', '703-333-1111', '804-222-3355'} |        Joe Buck
 cf11c639-102f-4812-8a78-4cb3b7028594 |     0057 |   Allissa | Abdelwahed | {'571-145-6698', '703-892-4565', '804-339-2555'} |     Priya Singh
 20100ad9-9dbe-451b-bede-956c8a9b20ef |     0056 |     Danny |       Boyd | {'572-265-6969', '703-888-0036', '804-555-1232'} |     Tucker Tank
 ec32987f-bfdd-4a40-ae2a-19dc4e1f04ce |     0057 |     Rebel |     Wilson | {'572-777-8898', '703-123-2244', '804-111-2235'} | Sherry Williams
 8de03854-afa7-4525-a43a-40dcefe782fc |     0056 |     Manny |      Aboah | {'572-125-8898', '703-895-4563', '804-569-2235'} | Sherry Williams
 
(5 rows)

 # Show data from offices table
 cqlsh> select * from hw07.offices;

 officeid | officename       | city          | streetaddress
----------+------------------+---------------+-----------------------------------------------
     0057 |           AMZ-10 |        Reston |           1777 Worldgate Dr. Reston, VA 20171
     0056 | Deloitte Rosslyn |       Rosslyn |           1777 N Kent St, Arlington, VA 22209
     0058 |             Vaco |  Falls Church | 2980 Fairview Park Dr, Falls Church, VA 22042
     0059 |  Deloitte Tysons | Tysons Corner |     7900 Tysons One Pl #800, McLean, VA 22102

(4 rows)

# Select with filters on primary keys from both tables
cqlsh> select * from hw07.employees where employeeid = 7689cb09-dc2a-4b09-9020-d75fcfae4baf;

 employeeid                           | officeid | firstname | lastname | phonenumbers                                     | supervisorname
--------------------------------------+----------+-----------+----------+--------------------------------------------------+----------------
 7689cb09-dc2a-4b09-9020-d75fcfae4baf |     0056 |     Johny |    Quest | {'571-125-8899', '703-333-1111', '804-222-3355'} |       Joe Buck

(1 rows)
cqlsh> select * from hw07.offices where officeid = '0056';

 officeid | officename       | city    | streetaddress
----------+------------------+---------+-------------------------------------
     0056 | Deloitte Rosslyn | Rosslyn | 1777 N Kent St, Arlington, VA 22209

# Select with filter on other field that is not a primary  key throws an error since it is inefficient - it would requier a full scan
cqlsh> select * from hw07.offices where officeName = 'AMZ-10';
InvalidRequest: Error from server: code=2200 [Invalid query] message="Cannot execute this query as it might involve data filtering and thus may have unpredictable performance. If you want to execute this query despite the performance unpredictability, use ALLOW FILTERING"

# Allow filering
cqlsh> select * from hw07.offices where officeName = 'AMZ-10' ALLOW FILTERING;

 officeid | officename | city   | streetaddress
----------+------------+--------+-------------------------------------
     0057 |     AMZ-10 | Reston | 1777 Worldgate Dr. Reston, VA 20171

(1 rows)
```

### Is there a way to impose referential integrity between table Employees and table Offices?

Ans: Since cassandra is a de-normalized database and does not allow table joins, <br>
the only way is to include office data such as officeid's and or office names in a single table.

### Could you, using a single CQL query, select information about an employee’s last and first name and the office in which he or she works

Ans: Cassandra does not allow table joins. So only officeId can be included in this query.

```sql
cqlsh> select lastname, firstname, officeid from hw07.employees where employeeid =  cf11c639-102f-4812-8a78-4cb3b7028594;
                     
 lastname   | firstname | officeid
------------+-----------+----------
 Abdelwahed |   Allissa |     0057
```

### Problem 3. 
Use Cassandra’s Python client(s) to create new table cosmonauts with columns: name text, year int, group int, dob text, spaceflight_hours int. Populate that table with data from the attached file astronauts.csv. Use composite PRIMARY KEY (group, name). Populate the table with all 357 astronauts, one way or the other. You might have to read Cassandra documentation to find the way how to implement this problem. 

```py
"""🐍 App that reads astronaut csv and loads it into cassadra table
"""

import csv
import logging

from cassandra.cluster import Cluster

# Log configurations
log = logging.getLogger()
log.setLevel('INFO')
handler = logging.StreamHandler()
handler.setFormatter(logging.Formatter("%(asctime)s [%(levelname)s] %(name)s: %(message)s"))
log.addHandler(handler)


KEYSPACE = "hw07_p3"

def main():
    # Connect to dse cluster
    cluster = Cluster()
    session = cluster.connect()

    # If keyspace already exists, drop it
    rows = session.execute("SELECT keyspace_name FROM system_schema.keyspaces")

    if KEYSPACE in [row[0] for row in rows]:
        log.info(f"dropping existing keyspace: {KEYSPACE}")
        session.execute("DROP KEYSPACE " + KEYSPACE)
    
    # Create the keyspace
    session.execute("""
        CREATE KEYSPACE %s
        WITH replication = {'class': 'SimpleStrategy', 'replication_factor': '2'}
        """ % KEYSPACE
        )
    
    # Set the keyspace
    log.info(f"Setting keyspace to {KEYSPACE}")
    session.set_keyspace(keyspace=KEYSPACE)
    
    # Create cosmonauts table
    log.info("creating astronauts table...")
    session.execute("""
        CREATE TABLE astronauts (
            name text, 
            year int, 
            group int,
            status text,
            dob text,
            birthplace text,
            gender text,
            alma_mater text,
            spaceflights int,
            spaceflight_hours int,
            spacewalks int,
            spacewalk_hours int,
            missions text,
            PRIMARY KEY (group, name)
        )
        """)
    
    prepared_stmnt = session.prepare("""
                INSERT INTO astronauts (name,year,group,status,dob,birthplace,
                gender,alma_mater,spaceflights,spaceflight_hours,spacewalks,spacewalk_hours,missions)
                VALUES (?, ?,?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                """)
    
    # Read csv file
    with open("astronauts.csv") as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=",")
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                log.info(f'Writing columns: {", ".join(row)}')
                line_count +=1
            else:
                log.info(f"""about to write line: \n
                {row[0]}, {row[1]}, 
                {row[2]}, {row[3]}, {row[4]}, 
                {row[5]}, {row[6]}, {row[7]}, 
                {row[8]}, {row[9]}, {row[10]}, 
                {row[11]}, {row[12]}"""
                )

                name = row[0]
                year = int(row[1])
                group = int(row[2])
                status = row[3]
                dob = row[4]
                birthplace = row[5]
                gender = row[6]
                alma_mater = row[7]
                spaceflights = int(row[8])
                spaceflight_hours = int(row[9])
                spacewalks = int(row[10])
                spacewalk_hours = int(row[11])
                missions = row[12]

                session.execute(prepared_stmnt,
                                [name, 
                                 year, 
                                 group, 
                                 status, 
                                 dob, 
                                 birthplace, 
                                 gender, 
                                 alma_mater, 
                                 spaceflights, 
                                 spaceflight_hours, 
                                 spacewalks, 
                                 spacewalk_hours, 
                                 missions]
                                )
                log.info(f'line: {line_count} processed')
                line_count +=1

    log.info(f"Finished processing {line_count} lines of astronaut data.")

    csv_file.close()
    
    session.shutdown()

# Main method
if __name__ == "__main__":
    main()
```

Show Data in cassandra:

```bash
cqlsh> select count(*) from hw07_p3.astronauts;

 count
-------
   357

(1 rows)

Warnings :
Aggregation query used without partition key

cqlsh> select * from hw07_p3.astronauts limit 10;

 group | name                  | alma_mater                        | birthplace       | dob        | gender | missions                                  | spaceflight_hours | spaceflights | spacewalk_hours | spacewalks | status   | year
-------+-----------------------+-----------------------------------+------------------+------------+--------+-------------------------------------------+-------------------+--------------+-----------------+------------+----------+------
     5 |      Alfred M. Worden |               US Military Academy |      Jackson, MI |   2/7/1932 |   Male |                                 Apollo 15 |               295 |            1 |               1 |          1 |  Retired | 1966
     5 |  Bruce McCandless II  |                  US Naval Academy |       Boston, MA |   6/8/1937 |   Male | STS 41-B (Challenger), STS-31 (Discovery) |               312 |            2 |              12 |          2 |  Retired | 1966
     5 |  Charles M. Duke Jr.  |                  US Naval Academy |    Charlotte, NC |  10/3/1935 |   Male |                                 Apollo 16 |               265 |            1 |              20 |          3 |  Retired | 1966
     5 |           Don L. Lind |                University of Utah |      Midvale, UT |  5/18/1930 |   Male |                     STS 51-B (Challenger) |               168 |            1 |               0 |          0 |  Retired | 1966
     5 |     Edgar D. Mitchell |        Carnegie-Mellon University |     Hereford, TX |  9/17/1930 |   Male |                                 Apollo 14 |               216 |            1 |               9 |          2 |  Retired | 1966
     5 | Edward G. Givens Jr.  |                  US Naval Academy |       Quanah, TX |   1/5/1930 |   Male |                                           |                 0 |            0 |               0 |          0 | Deceased | 1966
     5 |    Fred W. Haise Jr.  |            University of Oklahoma |       Biloxi, MS | 11/14/1933 |   Male |                                Apollo 13  |               142 |            1 |               0 |          0 |  Retired | 1966
     5 |        Gerald P. Carr | University of Southern California |       Denver, CO |  8/22/1932 |   Male |                                  Skylab 4 |              2017 |            1 |              16 |          3 |  Retired | 1966
     5 |        Jack R. Lousma |            University of Michigan | Grand Rapids, MI |  2/29/1936 |   Male |                Skylab 3, STS-3 (Columbia) |              1619 |            2 |              11 |          2 |  Retired | 1966
     5 |        James B. Irwin |                  US Naval Academy |   Pittsburgh, PA |  3/17/1930 |   Male |                                 Apollo 15 |               295 |            1 |              20 |          3 | Deceased | 1966

(10 rows)
```